## 单因子策略（一）

In [ ]:
#============================================================
# 单因子策略（一）
#在编写策略时，我们首先需要大致确定一下我们编写过程需要用到的几个Python包，比如numpy.pandas.datetime，您
#需要根据您的需求，在策略的最上方进行导包操作。
from datetime import timedelta, date
#导入datetime相关包
import pandas as pd
#导入pandas包，用于数据分析。
#==================================================================
#==================================================================

#首先我们需要一个策略初始化函数，其功能相当于开户。
def init(context):
#def是Python语言创建函数的标志，initialize是函数名称，这里是初始化函数，（context）是账户信息对象，也就是
#该初始化函数中，只有初始一些账户信息。并且该函数只运行一次，相当于开户只需要一次就可以了。
    context.n = 15 
    #设置n=15，并将其放到账户信息中，主要为了控制持股数量，控制到15个及以内。
    run_monthly(trade,date_rule=-1)
    #按月运行函数run_monthly（a,b），其中里面有两个参数，a是运行对象，这里是trade函数（详见34行代码），
    #b是运行时间，这里是每个月倒数第一个交易日。
#=======================================================================
#=======================================================================
#其次我们需要构建一个交易系统，使得整个策略能运行起来！
def trade(context, bar_dict):
#我们创建一个取名为trade的函数，用于交易，其中设置context参数，我们就可以到别的函数中的context对象中的信息
#用到这里来，另外我们还有bar_dict参数，用于计算数据。
    pb_list = stocks_pb(context,bar_dict)
    #我们将stocks_pb(context,bar_dict)函数的选股结果导入pb_list对象
    stock_list = list(set(pb_list))
    #我们需要将pb_list对象的格式进行转化，使得运行不出错，直接用list(set(x))即可。其中set()是创建集合
    #这里是创建股票集合，list()用来创建列表，这里是创建股票列表。
    if len(list(context.portfolio.stock_account.positions.keys())) > 0:
    #if 判断函数，len()是取长度数值，context.positions是账户持仓信息，len(context.positions)也就是
    #当前持仓股票的数量，如果大于0，则进行下一步。
        for stock in list(context.portfolio.stock_account.positions.keys()):
        #for x in y 是循环函数，从y中逐一取出来设为x，这里的y是持仓个股列表。x是stock，将个股取出，
        #stock中存放取出的个股，执行下一步，如果一个个股执行完毕，执行下一个个股，直到账户中所有持仓
        #个股全部被执行后，跳出循环。
            if stock not in stock_list:
            #if判断函数，判断选股股票是否在选股函数列表，不在则运行下一步
                order_target(stock, 0)
                #order_target下单函数，stock是买卖的股，0是目前持仓，也就是清仓。
    if len(stock_list) > 0:
    #if 判断函数，判断选股列表中股票数量是否大于0
        for stock in stock_list:
        #for x in y循环函数，从股票列表中逐一取出个股
            if stock not in list(context.portfolio.stock_account.positions.keys()):
            #if判断函数，用于判断个股是否在个股持仓列表中。
                if len(list(context.portfolio.stock_account.positions.keys())) < context.n :
                #if 判断函数，用于判断持仓个股数，是否小于我们初始化设置的条件15个。
                    number = context.n  - len(list(context.portfolio.stock_account.positions.keys()))
                    #如果不超出，那么计算，我们还能买的个股数量
                    order_value(stock,context.portfolio.stock_account.available_cash/number)
                    #下单函数，执行买入，买入的金额为可用现金/可买个股数
                else: 
                    order_value(stock,context.portfolio.stock_account.available_cash)
                #else是if函数的衍生，如果个股数量等于或超过最大持仓，那么我们将剩下的资金全部买入

#======================================================================
#我们需要创建一个函数，其作用是选择我们股票，因子是pb市净率，来让我们的交易系统有股票可以买卖
def stocks_pb(context,bar_dict):
#我们创建一个取名为stocks_pb的函数，用于选股，其中设置context参数，我们就可以到别的函数中的context对象
#中的信息用到这里来，另外我们还有bar_dict参数，用于计算数据。
    last_date = get_last_datetime().strftime('%Y%m%d')
    #get_last_datetime()函数用来获取回测的前一天的日期，后面的strftime('%Y%m%d')是用来转化成年月日的格式。
    pb = get_fundamentals(query(valuation.symbol,valuation.pb
                        ).filter(valuation.pb > 0,
                        ).order_by(valuation.pb.asc()
                        ),date = last_date)
    #get_fundamentals是获取财务数据的函数，query(x)是获取对象，这里是股票和pb，filter(y)是用来筛选条件，这
#里是pb>0，order_by(z)用来排序，将选择后的个股排序，这里是pb从小到大，date是日期，这里是回测前一日。
#其中sec()是从小到大排序。
    return list(pb['valuation_symbol'][:15])
    #return用来输出函数的执行结果。这里是我们选择出来的股票列表的前15只个股。
    
    
    